# Software documentation

1. Introduction
2. Data description
3. Software organization
4. Functioning

## 1.Introduction

Our goal is to build a software that enables to populate and to query two kind of databases: a relational and a graph database.  
We will store datasets, coming from CSV and JSON files, into our databases, as well as providing the possibility to query these databases simultaneously by having in return specific lists of Python objects.  
We started by analysing the data that has been provided in order to understand the different cases we had to handle and their characteristics.  We then created two data processors that we will use to extract data from the datasets provided and we prepare the envioroment to create from them tables, for del relational database, and triples, for the graph one.

---


## 2.Data description

We started by analyzing the exemplar JSON and CSV files that we have been provided with to test the software.
The CSV files, both the relational_publications.csv and the graph_publications.csv, are composed by the following columns:

<code><strong>id, title, type, publication_year, issue, volume, chapter, publication_venue, venue_type, publisher, event</strong></code>

Each row defines a publication entity.
As defined in the given UML, we will have thre type of pubblications: journal articles, book chapters and proceedings papers.
Journal articles can also have issu and volume specified, while book chapters must have a chapter number.

In the JSON files we will find additional informations about the publications and their related classes. In particular the JSON files are structured in 4 main keys:

<code><strong> authors, venues_id, references, publishers </strong></code>

The first three sections contain additional information about authors, venues and citations related publication by means of the publications' unique identifiers (DOI) used as sub-key inside each of these 4 macro "dictionaries". The fourth key give further information about the publishers that can be connected to the inforamtion of our csv through the mediation of their crossref identifier, which is used as key inside the json.

The files have been analysed both manually and by means of Python in order to better understand both the quantitative and the qualitative characteristics fo the data.

### Quantitative characteristcs of datasets.

#### CSV

In [22]:
from pandas import *

relational_publications = read_csv("data/relational_publications.csv")
relational_publications.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 500 non-null    object 
 1   title              500 non-null    object 
 2   type               500 non-null    object 
 3   publication_year   500 non-null    int64  
 4   issue              347 non-null    object 
 5   volume             443 non-null    object 
 6   chapter            22 non-null     float64
 7   publication_venue  498 non-null    object 
 8   venue_type         498 non-null    object 
 9   publisher          498 non-null    object 
 10  event              0 non-null      float64
dtypes: float64(2), int64(1), object(8)
memory usage: 43.1+ KB


In [23]:
relational_publications.head()

,id,title,type,publication_year,issue,volume,chapter,publication_venue,venue_type,publisher,event
0,doi:10.1162/qss_a_00023,"Opencitations, An Infrastructure Organization ...",journal-article,2020,1,1,NaN,Quantitative Science Studies,journal,crossref:281,NaN
1,doi:10.1007/s11192-019-03217-6,"Software Review: Coci, The Opencitations Index...",journal-article,2019,2,121,NaN,Scientometrics,journal,crossref:297,NaN
2,doi:10.1007/s11192-019-03311-9,Nine Million Book Items And Eleven Million Cit...,journal-article,2019,2,122,NaN,Scientometrics,journal,crossref:297,NaN
3,doi:10.1038/sdata.2016.18,The Fair Guiding Principles For Scientific Dat...,journal-article,2016,1,3,NaN,Scientific Data,journal,crossref:297,NaN
4,doi:10.1371/journal.pbio.3000385,The Nih Open Citation Collection: A Public Acc...,journal-article,2019,10,17,NaN,Plos Biology,journal,crossref:340,NaN


In [24]:
graph_publication = read_csv("data/graph_publications.csv")
graph_publication.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 500 non-null    object 
 1   title              500 non-null    object 
 2   type               500 non-null    object 
 3   publication_year   500 non-null    int64  
 4   issue              303 non-null    object 
 5   volume             391 non-null    object 
 6   chapter            93 non-null     float64
 7   publication_venue  486 non-null    object 
 8   venue_type         486 non-null    object 
 9   publisher          486 non-null    object 
 10  event              0 non-null      float64
dtypes: float64(2), int64(1), object(8)
memory usage: 43.1+ KB


In [25]:
graph_publication.head()

,id,title,type,publication_year,issue,volume,chapter,publication_venue,venue_type,publisher,event
0,doi:10.1016/j.websem.2021.100655,Crossing The Chasm Between Ontology Engineerin...,journal-article,2021,NaN,70,NaN,Journal Of Web Semantics,journal,crossref:78,NaN
1,doi:10.1007/s10115-017-1100-y,Core Techniques Of Question Answering Systems ...,journal-article,2017,3,55,NaN,Knowledge And Information Systems,journal,crossref:297,NaN
2,doi:10.1016/j.websem.2014.03.003,Api-Centric Linked Data Integration: The Open ...,journal-article,2014,NaN,29,NaN,Journal Of Web Semantics,journal,crossref:78,NaN
3,doi:10.1093/nar/gkz997,The Monarch Initiative In 2019: An Integrative...,journal-article,2019,D1,48,NaN,Nucleic Acids Research,journal,crossref:286,NaN
4,doi:10.3390/publications7030050,Dras-Tic Linked Data: Evenly Distributing The ...,journal-article,2019,3,7,NaN,Publications,journal,crossref:1968,NaN


As we see in this first exploration of the two CSV provided, we already see some quantitative difference between the two datasets. This lead to the conclusion that the two databases we will create cuold have different informations about different publications.

#### JSON

In [26]:
relational_other_data = read_json("data/relational_other_data.json")
relational_other_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 540 entries, doi:10.1162/qss_a_00023 to crossref:301
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   authors     508 non-null    object
 1   venues_id   498 non-null    object
 2   references  500 non-null    object
 3   publishers  32 non-null     object
dtypes: object(4)
memory usage: 21.1+ KB


In [27]:
relational_other_data.describe(include="all")

,authors,venues_id,references,publishers
count,508,498,500,32
unique,486,297,99,32
top,"[{'family': 'Leydesdorff', 'given': 'Loet', 'o...","[issn:0138-9130, issn:1588-2861]",[],"{'id': 'crossref:6228', 'name': 'Codon Publica..."
freq,4,50,366,1


In [28]:
graph_other_data = read_json("./data/graph_other_data.json")
graph_other_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 563 entries, doi:10.1016/j.websem.2021.100655 to crossref:4443
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   authors     526 non-null    object
 1   venues_id   486 non-null    object
 2   references  500 non-null    object
 3   publishers  37 non-null     object
dtypes: object(4)
memory usage: 22.0+ KB


In [29]:
graph_other_data.describe(include="all")

,authors,venues_id,references,publishers
count,526,486,500,37
unique,492,309,101,37
top,"[{'family': 'Pal', 'given': 'Kamalendu', 'orci...",[issn:2076-3417],[],"{'id': 'crossref:735', 'name': 'Thomas Telford..."
freq,4,15,331,1


As for the CSVs, also for the data provided inside the JSONs, we found some quantitative difference.

## 3.Software organization

We organized the software in order to be scalable and accessible.
It is scalable since we made a huge use of classes, and relative their methods, in order to keep the code as much general purpose as possible. The idea is to have a basic structure that can be adapted to different datasets.
It is accessible since we design an entry point (main.py) for the software. This feature was also important in the testing and debugging fases.
